In [2]:
import ROOT
import numpy as np
from src.histogram import Histogram1D

ROOT.TH1.SetDefaultSumw2()  # Sets weighted binning in all ROOT histograms by default
ROOT.TH1.AddDirectory(False)  # disables TH1 overwrite warning
ROOT.gROOT.SetBatch()  # Prevents TCanvas popups

a = np.random.normal(0, 1, 1000)
b = np.random.normal(0, 1.1, 1000)
w = np.random.random(1000)
nbins = 20
xmin = -1
xmax = 1

Welcome to JupyROOT 6.24/02


In [3]:
h = Histogram1D(a, (nbins, xmin, xmax), weight=w)
h2 = Histogram1D(b, (nbins, xmin, xmax), weight=w)
print(h)
print(h2)

                   ┌─────────────────────────────────────────────────────────┐
[-inf,   -1) 79.05 │██████████████████████████████████████████████████████▍  │
[  -1, -0.9) 11.91 │████████▎                                                │
[-0.9, -0.8) 21.47 │██████████████▊                                          │
[-0.8, -0.7) 19.35 │█████████████▍                                           │
[-0.7, -0.6) 14.08 │█████████▊                                               │
[-0.6, -0.5) 15.77 │██████████▉                                              │
[-0.5, -0.4) 18.56 │████████████▊                                            │
[-0.4, -0.3) 18.65 │████████████▉                                            │
[-0.3, -0.2) 18.55 │████████████▊                                            │
[-0.2, -0.1) 17.59 │████████████▏                                            │
[-0.1,    0) 16.61 │███████████▍                                             │
[   0,  0.1) 24.25 │████████████████▊               

In [4]:
h.chi_square_fit(h2)

(1.3942848583233913, 0.11706876104690082)

In [8]:
h.to_TH1().Chi2Test(h2.to_TH1(), "WW")

0.11706876104690082